In [1]:
import matplotlib.pyplot as plt
import numpy as np            
import pandas as pd        

from tensorflow import keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.applications import ResNet50
from keras.layers import Flatten, Dense, BatchNormalization, GlobalAveragePooling2D, Dropout
from keras.models import Model
from keras.callbacks import EarlyStopping, ReduceLROnPlateau
from sklearn.utils.class_weight import compute_sample_weight

#Đường dẫn thư mục train và test
train_path = r'.\train'
test_path = r'.\test'
val_path = r'.\val'

batch_size = 16 

img_height = 500
img_width = 500

In [2]:
image_gen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    rotation_range=30,
    width_shift_range=0.2,
    height_shift_range=0.2,
    brightness_range=[0.8,1.2],
    fill_mode='nearest'
)

test_data_gen = ImageDataGenerator(rescale=1./255)

In [3]:
train_generator = image_gen.flow_from_directory(
    train_path,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='binary',
    color_mode='rgb'
)

validation_generator = image_gen.flow_from_directory(
    val_path,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='binary',
    color_mode='rgb'
)

test = test_data_gen.flow_from_directory(
    test_path,
    target_size=(img_height, img_width),
    color_mode='rgb', 
    shuffle=False,
    class_mode='binary',
    batch_size=batch_size
)


Found 1943 images belonging to 2 classes.
Found 497 images belonging to 2 classes.
Found 611 images belonging to 2 classes.


In [4]:
base_model = ResNet50(include_top=False, input_shape=(img_height, img_width, 3))

# Đóng băng các lớp trong mô hình gốc
for layer in base_model.layers[:-10]:
    layer.trainable = False

# Thêm các lớp tùy chỉnh lên trên mô hình gốc
x = Flatten()(base_model.output)
x = Dense(128, activation='relu')(x)
x = BatchNormalization()(x)
x = Dropout(0.5)(x)  # Add dropout layer
x = Dense(64, activation='relu')(x)
x = BatchNormalization()(x)
x = Dropout(0.5)(x)  # Add dropout layer
predictions = Dense(1, activation='sigmoid')(x)

# Tạo mô hình
cnn = Model(inputs=base_model.input, outputs=predictions)
cnn.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Định nghĩa callbacks
early = EarlyStopping(monitor="val_loss", mode="min", patience=3)
learning_rate_reduction = ReduceLROnPlateau(monitor='val_loss', patience=2, verbose=1, factor=0.3, min_lr=0.000001)
callbacks_list = [early, learning_rate_reduction]

# Tính trọng số lớp
unique_classes = np.unique(train_generator.classes)
weights = compute_sample_weight(class_weight='balanced', y=train_generator.classes)
cw = dict(zip(unique_classes, weights))
print(cw)

# Huấn luyện mô hình
cnn.fit(
    train_generator,
    epochs=25,
    validation_data=validation_generator,
    class_weight=cw,
    callbacks=callbacks_list
)

# Lưu mô hình
cnn.save('resnet50_custom.h5')

{0: 1.763157894736842, 1: 1.763157894736842}
Epoch 1/25
122/122 [==============================] - 656s 5s/step - loss: 1.6000 - accuracy: 0.5512 - val_loss: 1.1196 - val_accuracy: 0.7082 - lr: 0.0010
Epoch 2/25
122/122 [==============================] - 683s 6s/step - loss: 1.3385 - accuracy: 0.6083 - val_loss: 0.8265 - val_accuracy: 0.7103 - lr: 0.0010
Epoch 3/25
122/122 [==============================] - 810s 7s/step - loss: 1.2868 - accuracy: 0.6444 - val_loss: 1.2403 - val_accuracy: 0.7103 - lr: 0.0010
Epoch 4/25
122/122 [==============================] - ETA: 0s - loss: 1.2300 - accuracy: 0.6547
Epoch 4: ReduceLROnPlateau reducing learning rate to 0.0003000000142492354.
122/122 [==============================] - 1053s 9s/step - loss: 1.2300 - accuracy: 0.6547 - val_loss: 1.2230 - val_accuracy: 0.7103 - lr: 0.0010
Epoch 5/25
122/122 [==============================] - 1160s 10s/step - loss: 1.1909 - accuracy: 0.6722 - val_loss: 0.6060 - val_accuracy: 0.7103 - lr: 3.0000e-04
Epoch 6

c:\Users\admin\AppData\Local\Programs\Python\Python38\lib\site-packages\keras\src\engine\training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [5]:
from sklearn.metrics import confusion_matrix, classification_report

# Evaluate the model on the test data
test_accu = cnn.evaluate(test)
print('The testing accuracy is:', test_accu[1] * 100, '%')

# Predict the labels for the test data
y_pred = cnn.predict(test)
y_pred = np.round(y_pred).astype(int)  # Chuyển đổi dự đoán thành nhãn nhị phân (0 hoặc 1)

# Get the true labels
y_true = test.classes

# Compute the confusion matrix
conf_matrix = confusion_matrix(y_true, y_pred)
print('Confusion Matrix:')
print(conf_matrix)

# Compute classification report
class_report = classification_report(y_true, y_pred, target_names=['Normal', 'Disease'])
print('Classification Report:')
print(class_report)

# Extract Sensitivity, Specificity, and F1-Score from confusion matrix
tn, fp, fn, tp = conf_matrix.ravel()
sensitivity = tp / (tp + fn)
specificity = tn / (tn + fp)
precision = tp / (tp + fp)
recall = sensitivity  # Recall là tên khác của Sensitivity
f1_score = 2 * (precision * recall) / (precision + recall)

print(f'Sensitivity: {sensitivity * 100:.2f}%')
print(f'Specificity: {specificity * 100:.2f}%')
print(f'F1-Score: {f1_score:.2f}')

39/39 [==============================] - 113s 3s/step - loss: 0.6298 - accuracy: 0.6776
The testing accuracy is: 67.75777339935303 %
39/39 [==============================] - 238s 6s/step
Confusion Matrix:
[[  0 197]
 [  0 414]]
Classification Report:
              precision    recall  f1-score   support

      Normal       0.00      0.00      0.00       197
     Disease       0.68      1.00      0.81       414

    accuracy                           0.68       611
   macro avg       0.34      0.50      0.40       611
weighted avg       0.46      0.68      0.55       611

Sensitivity: 100.00%
Specificity: 0.00%
F1-Score: 0.81


c:\Users\admin\AppData\Local\Programs\Python\Python38\lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\admin\AppData\Local\Programs\Python\Python38\lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\admin\AppData\Local\Programs\Python\Python38\lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, mo